# Init

In [5]:
import time, sys
%load_ext autoreload
%autoreload 2

## Custom tools

In [6]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.readdata as readdata

In [289]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.filtering import filter2

In [8]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb

In [135]:
i_CFG = 0
i_RD = 1
i_CAL = 2
i_f = 3

i_x = 2
i_y = 3
i_z = 4

# antenna locations on the plate
Dx1   = 0.224
Dx2   = 0.168
h     = 0.04
# positions of the box in x
xMinTrail  = -1.50
xMaxTrail  = 1.488074688911438
# bizona
x_biz_offset = 0
y_biz_offset = 6.4
z_biz_offset = 3
biz_offset = (x_biz_offset, y_biz_offset, z_biz_offset)

In [10]:
from backprojection.loadbackprojection import *
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"
lib = LibBackProjection( libraryFilename )

## Numpy

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [12]:
%matplotlib qt

In [152]:
cmap='jet'

## Data paths

In [20]:
day = "06_03_2017"
base_path = "/home/pleroy/DATA/DIADEM/lff" \
    + f"/fromLHA/Mesure_prepa_Hambourg/Mesures_traitées/mesure_bistatique_originales_{day}_ple"
prefix = "BIST_VVVV"
nbDir = 5
RxTx = ["21", "31", "32", "41", "42", "43"]

In [21]:
heights = {}
for k in range(1, nbDir+1):
    filename = base_path + f"/{prefix}_{k}/PoSAR.cfg"
    CFG = gb.readCfg( filename )
    heights[f"{k}"] = CFG["H_nom"]

In [22]:
heights

{'1': 1.71, '2': 1.59, '3': 1.47, '4': 1.35, '5': 1.23}

# Read separate files

## Read configuration

In [24]:
CFG = gb.readCfg( f"{base_path}/{prefix}_1/PoSAR.cfg" )

In [25]:
CFG

{'H_nom': 1.71,
 'xamin_nom': -1.5,
 'xamax_nom': 1.488075,
 'Npos': 250,
 'theta_c': 0.7853981633974483,
 'theta_ap': 0.7853981633974483,
 'phi_c': 0.0,
 'phi_ap': 0.7853981633974483,
 'Fmin': 8399999488.0,
 'Fmax': 12400000000.0,
 'Nf': 401,
 'c': 300000000.0}

In [242]:
Npos = CFG['Npos']

## Read data

In [94]:
ij = '21'
c = gb.readFile( f"{base_path}/{prefix}_1/S{ij}.dat", CFG['Npos'], CFG['Nf'] )

In [28]:
fix, (ax0, ax1) = plt.subplots(1,2)

im = ax0.imshow(20 * np.log10( np.abs( c ) ) )
dia.addColorBar(im, ax0)

im = ax1.imshow( np.unwrap( np.angle(c), axis = 1 ) )
dia.addColorBar(im, ax1)

In [29]:
angle = np.angle(c[125,:])
fig, ax = plt.subplots(1,1)
ax.plot( angle, '.-' )
ax.plot( np.unwrap(angle) )
ax.grid()

## Read positions

In [34]:
data_idx = 1
ij = "21"
ant_pos = gb.readPos( f"{base_path}/{prefix}_{data_idx}/track_S{ij}.dat" )

In [35]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos.Txx, '.-', label=f"Txx")
ax.plot(ant_pos.Txy, '.-', label=f"Txy")
ax.plot(ant_pos.Txz, '.-', label=f"Txz")
ax.plot(ant_pos.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos.Rxz, '.-', label=f"Rxz")
ax.legend()
ax.grid()
title = f"{day} {prefix} {ij}"
ax.set_title(title)

Text(0.5, 1.0, '06_03_2017 BIST_VVVV 21')

# read_PoSAR_data

In [276]:
vmax=-60
vmin=-90

In [350]:
data_idx = 1
ij = "41"
z3 = heights[f"{data_idx}"]
BIST1 = gb.readData( f"{base_path}/{prefix}_{data_idx}", f"S{ij}.dat" )

In [351]:
f_base = BIST1[i_f] - BIST1[i_CFG]["Fmin"]
r = np.arange(f_base.size) * BIST1[i_CFG]["delta_sr"]
az = (ant_pos.Txx + ant_pos.Txz)/2

In [352]:
hanning = np.hanning( BIST1[i_CFG]["Nf"] )
coupling_RD = np.average( BIST1[i_RD], 0 )
ifft_RD                 = np.fft.ifft( BIST1[i_RD] )
ifft_RD_withHanning     = np.fft.ifft( BIST1[i_RD] * hanning )
ifft_RD_withoutCoupling = np.fft.ifft( BIST1[i_RD] - coupling_RD )

In [353]:
fig, ax = plt.subplots(1,1)

im = ax.plot( r, 20 * np.log10( np.abs( coupling_RD ) ), label='coupling' )
im = ax.plot( r, 20 * np.log10( np.abs( ifft_RD[int(Npos/2)] ) ), label=f'[{ij}] ifft_RD z3={z3}' )
ax.grid()
ax.legend()

In [277]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD ) ), cmap='jet',vmin=vmin,vmax=vmax )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle( f"{day} {prefix} [{ij}] z3={z3} raw data" )

Text(0.5, 0.98, '06_03_2017 BIST_VVVV [31] z3=1.71 raw data')

In [128]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD_withoutCoupling ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD_withoutCoupling ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle( f"{day} {prefix} [{ij}] without coupling" )

Text(0.5, 0.98, '06_03_2017 BIST_VVVV [41] without coupling')

In [233]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD_withHanning ) ), cmap='jet', vmax=vmax,vmin=vmin  )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD_withHanning ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle( f"{day} {prefix} [{ij}] with hanning" )

Text(0.5, 0.98, '06_03_2017 BIST_VVVV [31] with hanning')

# Build positions

In [69]:
Npos = CFG["Npos"]
zRail   = gb.getZRail( "147cm", units="cm" )
xyz_dic = gb.buildPos( xMinTrail, xMaxTrail, Npos, Dx1, Dx2, h, zRail )
xyz_dicUsingTrack = gb.buildPosUsingTrack( f"{base_path}/{prefix}_1/track_S" , Dx1, Dx2, h, zRail, verbose=1 )
ant_pos21 = gb.readPos( f"{base_path}/{prefix}_1/track_S21.dat" )
ant_pos43 = gb.readPos( f"{base_path}/{prefix}_1/track_S43.dat" )

read 250 positions in file:
/home/pleroy/DATA/DIADEM/lff/fromLHA/Mesure_prepa_Hambourg/Mesures_traitées/mesure_bistatique_originales_06_03_2017_ple/BIST_VVVV_1/track_S21.dat


In [70]:
(Dx1+Dx2+Dx2)/2+np.amax(ant_pos21.Txx)

1.488074688911438

In [73]:
fig, ax = plt.subplots(1,1)
for a in ["1", "2", "3", "4"]:
    ax.plot( xyz_dic[a][:,i_x], xyz_dic[a][:,i_z], 'o', label=f"antenna {a}, xyz_dic" )
    ax.plot( xyz_dicUsingTrack[a][:,i_x], xyz_dicUsingTrack[a][:,i_z], 'x', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( ant_pos21.Txx, ant_pos21.Txz, '.', label="antenna 1, ant_pos read positions" )
ax.plot( ant_pos21.Rxx, ant_pos21.Rxz, '.', label="antenna 2, ant_pos read positions" )
ax.plot( ant_pos43.Txx, ant_pos43.Txz, '.', label="antenna 3, ant_pos read positions" )
ax.plot( ant_pos43.Rxx, ant_pos43.Rxz, '.', label="antenna 4, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )
ax.set_xlabel("x")
ax.set_ylabel("z")

Text(0, 0.5, 'z')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( ant_pos.Txz, 'o', label="antenna 1, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

In [ ]:
ant_pos_147_21 = gb.readPos( path_147 + "/track_S21.dat", CFG["Npos"] )
ant_pos_159_21 = gb.readPos( path_159 + "/track_S21.dat", CFG["Npos"] )
ant_pos_171_21 = gb.readPos( path_171 + "/track_S21.dat", CFG["Npos"] )

z3 = gb.getZRail("147cm", units="cm")
xyz_147 = gb.buildPosUsingTrack( path_147 + "/track_S" , Dx1, Dx2, h, z3 )
z3 = gb.getZRail("159cm", units="cm")
xyz_159 = gb.buildPosUsingTrack( path_159 + "/track_S" , Dx1, Dx2, h, z3 )
z3 = gb.getZRail("171cm", units="cm")
xyz_171 = gb.buildPosUsingTrack( path_171 + "/track_S" , Dx1, Dx2, h, z3 )

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot( ant_pos_147_21.Txx, ant_pos_147_21.Txz, '.', label="antenna 1, ant_pos_147" )
ax.plot( ant_pos_159_21.Txx, ant_pos_159_21.Txz, '.', label="antenna 1, ant_pos_159" )
ax.plot( ant_pos_159_21.Txx, ant_pos_171_21.Txz, '.', label="antenna 1, ant_pos_171" )

ax.plot( xyz_147["1"][:,i_x], xyz_147["1"][:,i_z], 'x', label="antenna 1, xyz_147" )
ax.plot( xyz_159["1"][:,i_x], xyz_159["1"][:,i_z], 'x', label="antenna 1, xyz_159" )
ax.plot( xyz_171["1"][:,i_x], xyz_171["1"][:,i_z], 'x', label="antenna 1, xyz_159" )

ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

# Define geometry

In [330]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 7
zMin = 0.
zMax = 0.

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)

nbX = x.size
nbY = y.size
nbZ = z.size

In [331]:
x.shape, y.shape, z.shape

((301,), (701,), (1,))

In [332]:
print( f"center of the scene ( {np.mean(x):.2f}, {np.mean(y):.2f}, {np.mean(z):.2f} )")

center of the scene ( 0.00, 3.50, 0.00 )


# Focalization

## Backprojection

In [333]:
lib.reload()

In [345]:
data_idx = 5
ij = "31"
path_xxx = f"{base_path}/{prefix}_{data_idx}"
d_shift = 0.20
overSamplingRatio = 10

read = gb.readData( path_xxx, f"S{ij}.dat" )
sr = np.fft.ifftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
rangeResolution = read[i_CFG]["delta_sr"]
Naz = sr.shape[0]
Nf  = sr.shape[1]

Rx, Tx = ij[0], ij[1]
z3 = heights[f"{data_idx}"]
xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z3, bizona=biz_offset )
xyzAntI = xyz_dic[ Rx ]
xyzAntJ = xyz_dic[ Tx ]

Nover   = overSamplingRatio * Nf
r_base  = np.arange( Nf ) * rangeResolution - d_shift
r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
dr_over = r_over[1] - r_over[0]

print( f"Nf = {Nf}, Naz = {Naz}" )
print( f"range from {r_over[0]:.2f}m to {r_over[-1]:.2f}m, "
      + f"resolution = {rangeResolution:.3e}m, oversampled = {rangeResolution / overSamplingRatio:.3e}m" )

Nf = 400, Naz = 250
range from -0.20m to 14.80m, resolution = 3.750e-02m, oversampled = 3.750e-03m


In [346]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.Nover     = r_over.size
myParameters.dx        = dr_over
myParameters.Naz       = Naz
myParameters.Nf        = Nf
myParameters.phi_a_deg = 60 # 20° => 7.4cm
myParameters.kc        = read[i_CFG]["kc"]

In [347]:
t = time.time()

focusedImg_a  = focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
print( "focusedImg_a.shape = {}".format( focusedImg_a.shape ) )

lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                               y,
                                               z,
                                               r_over,
                                               sr.reshape(sr.size),
                                               xyzAntI.reshape(xyzAntI.size),
                                               xyzAntJ.reshape(xyzAntJ.size),
                                               focusedImg_a.reshape(focusedImg_a.size),
                                               myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg_a.shape = (301, 701, 1)
execution time = 0.6629352569580078


## Plot focused images

In [348]:
Nfilt = 5
filt = np.ones( (Nfilt, Nfilt) )
filt = filt / np.sum(filt)

### focusedImg_a

In [349]:
toPlot = focusedImg_a[:,:,0]

fig, ax = plt.subplots(2,1, sharex=True, sharey=True)

im = ax[0].pcolormesh(y, x, 20 * np.log10( np.abs( toPlot ) ), cmap=cmap)
dia.addColorBar( im, ax[0], aspect="equal" )
ax[0].set_title( f"{day} [{ij}] z3={z3} d_shift={d_shift}" )

#filter2( filt, np.angle(toPlot) )
im = ax[1].pcolormesh(y, x, np.angle(toPlot), cmap=cmap)
dia.addColorBar( im, ax[1], aspect="equal" )
ax[1].set_title( f"{day} [{ij}] z3={z3} d_shift={d_shift}" )

Text(0.5, 1.0, '06_03_2017 [31] z3=1.23 d_shift=0.2')

# Process all

In [ ]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.phi_a_deg = 60 # 10° => 4.3cm

In [ ]:
d_shift = 0.2
overSamplingRatio = 10

img_h = {}

print( f"focusedImg.shape = ({nbX}, {nbY}, {nbZ})")

for height in heights:
    
    path_xxx = prefix + height
    z3 = gb.getZRail( height, units="cm" )
    xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z3, verbose=1 )
    print( f"height = {height}, z3 = {z3}" )
    
    img_ij = {}
    
    for ij in RxTx:

        Rx, Tx = ij[0], ij[1]
        xyzRx = xyz_dic[ Rx ]
        xyzTx = xyz_dic[ Tx ]
        hAver = ( xyzRx[0,i_z] + xyzTx[0,i_z] ) / 2
    
        read = gb.readData( path_xxx, f"S{ij}.dat" )
        
        sr = np.fft.fftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
        rangeResolution = read[i_CFG]["delta_sr"]
        Naz = sr.shape[0]
        Nf  = sr.shape[1]
    
        Nover   = overSamplingRatio * Nf
        r_base  = np.arange( Nf ) * rangeResolution - d_shift
        r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
        dr_over = r_over[1] - r_over[0]
    
        myParameters.Nover     = r_over.size
        myParameters.dx        = dr_over
        myParameters.Naz       = Naz
        myParameters.Nf        = Nf
        myParameters.kc        = read[i_CFG]["kc"]
    
        t = time.time()
    
        focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
        
        lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                                       y,
                                                       z,
                                                       r_over,
                                                       sr.reshape(sr.size),
                                                       xyzRx.reshape(xyzRx.size),
                                                       xyzTx.reshape(xyzTx.size),
                                                       focusedImg.reshape(focusedImg.size),
                                                       myParameters)

        elapsed = time.time() - t
        print( f"hAver = {hAver:.2f} (Rx,Tx) = ({Rx},{Tx}) execution time = {elapsed:.1f}")
        img_ij[ij] = focusedImg
    img_h[height] = img_ij

In [ ]:
img_h["171cm"]["21"].strides

In [ ]:
img_h["171cm"]["21"].shape, img_h["171cm"]["21"].dtype

In [ ]:
301*301*16

## Plot

In [ ]:
opt = disp.OPTt()
opt.db = 1
opt.med_dyn = 100

In [ ]:
x0 = np.amin( np.where( x >= -0.8 ) )
z0 = np.amin( np.where( z >= 0 ) )

In [ ]:
img_h[h_asString]["21"][0,:,:].shape

In [ ]:
h_asString = "171cm"
Rx = "2"
Tx = "1"
toPlot = 20 * np.log10( np.abs( img_h[h_asString]["21"][x0,:,:] ) )

opt.title = f"Synthesized image: {h_asString}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( toPlot, opt, 0, 0, [yMin,yMax,zMin,zMax] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("auto")
ax.yaxis.set_ticks_position("left")

In [ ]:
h_asString = "171cm"
Rx = "2"
Tx = "1"
toPlot = 20 * np.log10( np.abs( img_h[h_asString]["21"][x0,:,:] ) ).T

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
h_asString = "171cm"
Rx = "2"
Tx = "1"
toPlot = img_h[h_asString]["21"][:,:,z0]

opt.title = f"Synthesized image: {h_asString}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( toPlot, opt, 0, 0, [yMin,yMax,xMax,xMin] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("auto")
ax.xaxis.set_ticks_position("bottom")
ax.yaxis.set_ticks_position("left")

In [ ]:
h_asString = "171cm"
Rx = "2"
Tx = "1"
toPlot = 20 * np.log10( np.abs( img_h[h_asString]["21"][:,:,z0] ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )
ax.invert_yaxis()

## Sum

In [ ]:
imgSum = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
imgSumC = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
amp = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
idxx = x0

for height in heights:
    
    path_xxx = prefix + height
    
    for ij in RxTx:
        imgSum +=         img_h[height][ij][idxx,:,:]
        amp    += np.abs( img_h[height][ij][idxx,:,:] )

In [ ]:
#imgSum /= amp
#toPlot = 20 * np.log10( np.abs( imgSum.T) )
toPlot = np.abs( imgSum.T)

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )